# Text processing on the Linux command line

In this notebook we learn how to perform simple text processing tasks by combining a set of tools available on __UNIX-like systems__ (such as Linux and Mac OS) using __pipes__.

For a very brief introduction to the Linux command line, including links to additional documentation, see this notebook:


[Introduction to the Linux command line](01a_Intro_to_Linux_command_line.ipynb)

To run the commands inside this notebook, you need to install the bash kernel for jupyter, e.g. like this:
```
pip install bash-kernel
python -m bash_kernel.install
```

In [1]:
export LC_ALL=C

Let's have a look at the two files we are going to work with:

In [2]:
head data/alice_tok.txt

CHAPTER
I
.

Down
the
Rabbit-Hole
Alice
was
beginning


In [3]:
head data/stopwords.txt

a
about
above
after
again
against
ain
all
am
an


We have a **tokenized** version of the novel _Alice's Adventures in Wonderland_ and a list of English **stopwords**, words that express some grammatical function that we often want to ignore in text processing applications. For more details on this and how we got here, see this notebook:

[Basics of Text Processing](01_Text_processing.ipynb)

`grep` is the command for matching regular expressions. Let's use it to find capitalized words.

The pipe symbol `|` means that the output of one command becomes the input of the next one:

In [4]:
grep -E '^[A-Z][a-z]+' data/alice_tok.txt | head

Down
Rabbit-Hole
Alice
Alice
So
White
Rabbit
There
Alice
Rabbit
grep: write error: Broken pipe


_You can ignore the occasional `Broken pipe` errors, it just means that a command in the pipeline was still writing output when the next one was already finished_

The `cat` command is often used at the beginning of a pipe, since all it does by default is send the contents of the file to the standard output

In [5]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | head

Down
Rabbit-Hole
Alice
Alice
So
White
Rabbit
There
Alice
Rabbit
grep: write error: Broken pipe
cat: write error: Broken pipe


Counting can be implemented as a combination of sorting and aggregation:

In [6]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | head -20

    397 Alice
    108 The
     74 Queen
     67 And
     64 It
     61 King
     57 Turtle
     56 Mock
     55 You
     55 Hatter
     55 Gryphon
     44 Rabbit
     43 What
     42 Duchess
     40 She
     40 Dormouse
     37 But
     33 There
     33 Oh
     32 March


Let's save this for later. The `sed` command can be used for regex-based search-and-replace, here we use it to get a more convenient format. Then we sort the lines alphabetically, later we'll see why.

In [7]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | sed 's/^ *\([0-9]*\) \(.*\)$/\2\t\1/' | sort | head

Ada	1
Adventures	2
Advice	1
After	6
Ah	5
Ahem	1
Alas	1
Alice	397
All	6
Allow	1


In [8]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | sed 's/^ *\([0-9]*\) \(.*\)$/\2\t\1/' | sort > data/ent_freqs.txt

Let's filter stopwords

In [9]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | head -50 | sed 's/^[ 0-9]*//' | head

Alice
The
Queen
And
It
King
Turtle
Mock
You
Hatter


In [10]:
cat data/alice_tok.txt | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | head -50 | sed 's/^[ 0-9]*//' | sort | tr [:upper:] [:lower:] | comm -13 data/stopwords.txt - | head

alice
bill
cat
caterpillar
come
dinah
dodo
dormouse
duchess
gryphon


Now let's find a way to get rid of the first words of sentences.

In [11]:
cat data/alice_tok.txt | sed 's/^$/@/' | tr '\n' ' ' | sed 's/ @ /\n/g' | cut -d' ' -f2- | tr ' ' '\n' | head

I
.
the
Rabbit-Hole
Alice
was
beginning
to
get
very
tr: write error: Broken pipe
tr: write error
cut: write error: Broken pipe


In [12]:
cat data/alice_tok.txt | sed 's/^$/@/' | tr '\n' ' ' | sed 's/ @ /\n/g' | cut -d' ' -f2- | tr ' ' '\n' | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | head -50 | sed 's/^[ 0-9]*//' | sort | tr [:upper:] [:lower:] | comm -13 data/stopwords.txt - | head -30

alice
bill
cat
caterpillar
come
dinah
dodo
dormouse
duchess
gryphon
hare
hatter
king
majesty
march
mock
mouse
oh
pigeon
queen
rabbit
soup
turtle
well
white
yes


Now we can add the frequencies from the saved file

In [13]:
cat data/alice_tok.txt | sed 's/^$/@/' | tr '\n' ' ' | sed 's/ @ /\n/g' | cut -d' ' -f2- | tr ' ' '\n' | grep -E '^[A-Z][a-z]+' | sort | uniq -c | sort -nr | head -50 | sed 's/^[ 0-9]*//' | sort | tr [:upper:] [:lower:] | comm -13 data/stopwords.txt - | sed 's/^./\u&/' | join - data/ent_freqs.txt | sort -k2 -nr

Alice 397
Queen 74
King 61
Turtle 57
Mock 56
Hatter 55
Gryphon 55
Rabbit 44
Duchess 42
Dormouse 40
Oh 33
March 32
Hare 31
Mouse 30
Caterpillar 27
Cat 26
Well 23
White 22
Come 20
Dinah 14
Bill 14
Soup 13
Dodo 13
Yes 12
Majesty 12
Pigeon 11


### Homework
*Ex. 1 and 2 are mandatory for the Data-oriented Programming Paradigms course.*

1. (3 points) Redo all steps (starting with the preprocessing and segmenation steps in the Python notebook) on the German version of this text in `data/alice_de.txt`. There may be a few small things that you have to change. You can also choose another language, as long as you can find a plain text version online. Try [Project Gutenberg](https://www.gutenberg.org/)!

1. (12 points) Redo the simple extraction of entities in this notebook, but include multi-word entities (we didn't find the Mock Turtle or the March Hare!). There may be many different ways to do this.

1. (optional, for bonus points) Could you take all of this a step further and extract an English-German dictionary of names in the book? It doesn't have to be fully correct. Again there might be several different approaches!